In [1]:
import re
import numpy as np
from collections import defaultdict


In [2]:
input = '''Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
'''

# with open('input') as f:
#     input = f.read()


In [3]:
vertices = set()
tunnels = defaultdict(lambda: -1)
valves = dict()


In [4]:
for line in input.splitlines():
    lineData = re.findall('Valve (\w\w) has flow rate=(\d+); tunnel.*valves? (.*)', line)
    valve = lineData[0][0]
    vertices.add(valve)
    flowRate = int(lineData[0][1])
    valves[valve] = flowRate
    for v in lineData[0][2].split(', '):
        vertices.add(v)
        tunnels[(valve, v)] = 1  # in minutes


In [5]:
i2v = sorted(list(vertices))  # index to vertex name
v2i = dict(zip(i2v, range(len(i2v))))  # vertex name to index
dist = np.full((len(i2v), len(i2v)), np.infty)


# Floyd–Warshall algorithm
edges = list(tunnels.keys())
for edge in edges:
    dist[v2i[edge[0]]][v2i[edge[1]]] = tunnels[edge]
for vertex in i2v:
    dist[v2i[vertex]][v2i[vertex]] = 0
for k in range(0, len(i2v)):
    for i in range(0, len(i2v)):
        for j in range(0, len(i2v)):
            dist[i][j] = min(
                dist[i][j],
                dist[i][k] + dist[k][j]
            )

# dist[i][j] is the shortest path from i to j


In [6]:
# bitmask for fast valve set comparison
def valve_bitmask(valves):
    bitmask = 0
    for v in valves:
        bitmask |= 1 << (v2i[v] + 1)
    return bitmask


In [7]:
MAX_TIME = 26

relevant_valves = set([v for v in valves if valves[v] > 0])


def dp(pos, time, pressure, visited):
    result = [(pressure, visited)]
    if time > MAX_TIME:
        return result
    
    remaining_valves = relevant_valves - set(visited)
    for v in remaining_valves:
        cost = dist[pos][v2i[v]] + 1
        benefit = valves[v] * (MAX_TIME - time - cost)
        if benefit < 1:
            continue
        
        result += dp(v2i[v], time + cost, pressure + benefit, visited + [v])
    return result


In [8]:
paths = dp(v2i['AA'], 0, 0, [])

max_meOnly = int(max(paths, key=lambda x: x[0])[0])

# optimize paths list:
#   * Filter paths.
#     Two individuals will be able to reduce more pressure than a single one. Thus,
#     all relevant paths should at least release half of the pressure of part 1.
#   * Use bitmask instead of list of strings.
paths = [
    (p[0], valve_bitmask(p[1]))
    for p in paths
    if p[0] > max_meOnly // 2
]


In [9]:
print(int(max(
    [
        me[0] + elephant[0]
        for me in paths
        for elephant in paths
        if not me[1] & elephant[1]
    ]
)))


1707
